In [1]:
import sys
import os
import re
import operator
import csv
import pandas as pd
from collections import defaultdict
from collections import OrderedDict

start = re.compile(r'^Address Before a Joint Session.*')
finish = re.compile(r'^NOTE:.*')
line_pat = re.compile('.*')
#Counting a 'true' word as anything 5 letters or longer
word_pat = re.compile(r'\b[a-zA-Z]{5,}')

In [2]:
def get_speeches():
    """ 
    Get a list of speeches in each directory
    """
    speeches = []
    for speech in os.listdir('.'):
        if speech[-4:] != '.txt':
            continue
        speeches.append(speech)

    return speeches

def all_words_to_string(speech):
    """
    For each speech we create a dictionary of all words and their counts.
    Then we just export the indexes/words to a single list.
    """
    
    dic = {}
    flag = False
    with open(speech, "r") as f:
        for line in f:
            if not flag:
                if start.match(line):
                    flag = True
                continue
            if finish.match(line):
                break
            result = line_pat.findall(line)
            result2 = word_pat.findall(line)
            for line in result:
                if result2:
                    for word in result2:
                        key1 = word.lower()
                        if key1 not in dic:
                            dic[key1] = 1
                        else:
                            dic[key1] += 1
    
    word_list = list(dic.keys())
    word_list = ' '.join(word_list)
    return word_list




In [3]:
categories = ['bush',
              'clinton',
              'obama',
              'trump']

In [4]:
data_dir = './data/train'
os.chdir(data_dir)
tup_list = []
for i in categories:
    os.chdir(i)
    speeches = get_speeches()
    for speech in speeches:
        word_list = all_words_to_string(speech)
        tup = (word_list, i)
        tup_list.append(tup)  
    os.chdir('..')
       
df = pd.DataFrame(tup_list, columns=['text','orator'])

In [6]:
df.to_csv('all_speeches.csv')